In [7]:
import backtrader as bt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [11]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import numpy as np
# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        t  = self.datas[0].datetime.time(0)
        #print('%s, %s' % (dt.isoformat(), txt))
        print('%s, %s, %s' % (dt.isoformat(),t, txt))

    def __init__(self):
        params = dict(period1=20,period2=45)
        # Keep a reference to the "close" line in the data[0] dataseries
        # Also, keeping a reference for volume 
        self.dataclose = self.datas[0].close
        self.datavolume = self.datas[0].volume
        
        self.movavl = btind.SimpleMovingAverage(self.dataclose, period=45)
        self.movavs = btind.SimpleMovingAverage(self.dataclose, period=20)
        
        # the problem of using the moving average as this one
        # will be that it be be a continous line
        # so rather try to have a filter 
        
        # Trying out the custom time 
        '''
        t  = self.datas[0].datetime.time(0)
        if t > datetime.time(10,0) :
            self.movavl = np.mean(self.dataclose[-46:-1])
            self.movavs = np.mean(self.dataclose[-21:-1])
        '''
        
        
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f at volume %d' % (self.dataclose[0],self.datavolume[0]))
        # modifying to log the close price of the day
        t  = self.datas[0].datetime.time(0)
        #self.log('Close-{:.2f},SMAl-{:.2f},SMAs-{:.2f}'.format(self.dataclose[0],self.movavl[-1],self.movavs[-1]))
        # Do remember to take -1 as SMA as these will be updated upto the previous value
        # taking 0 will create unbaised result.
        if t >= datetime.time(10,0) :
            if self.movavs[-1] > self.movavl[-1] :
                if self.movavs[-2] < self.movavl[-2]: 
                    self.log('B,MAs:{:.2f},MAl:{:.2f}'.format(self.movavs[-1],self.movavl[-1]))
            elif self.movavs[-1] < self.movavl[-1]:
                if self.movavs[-2] > self.movavl[-2]: 
                #self.log('S')
                    pass
            else:
            #self.log('H')
                pass
        
        
        '''
        if t == datetime.time(15, 30):
            self.log('Close @ eod , %.2f with volume @ eod %d ' % (self.dataclose[0],self.datavolume[0]))
        elif t== datetime.time(9,15):
            self.log('Close @ open , %.2f with volume @ open %d ' % (self.dataclose[0],self.datavolume[0]))
        '''

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
    #datapath='nse1.csv'
    # Create a Data Feed
    data = btfeeds.GenericCSVData(
        dataname='nse1.csv',
        timeframe=bt.TimeFrame.Minutes,
        fromdate=datetime.datetime(2018, 1, 1),
        todate=datetime.datetime(2018, 1, 4),
    nullvalue=0.0,

    dtformat=('%Y-%m-%d'),
    tmformat=('%H:%M:%S') ,   

    datetime=0,
    time=1,
    high=3,
    low=4,
    open=2,
    close=5,
    volume=6,
    openinterest=7
)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2018-01-01, 10:02:00, B,MAs:10543.91,MAl:10543.58
2018-01-01, 11:29:00, B,MAs:10552.42,MAl:10552.39
2018-01-01, 12:39:00, B,MAs:10550.79,MAl:10550.74
2018-01-01, 14:03:00, B,MAs:10540.19,MAl:10540.02
2018-01-02, 11:06:00, B,MAs:11081.60,MAl:11081.50
2018-01-02, 13:09:00, B,MAs:10989.18,MAl:10985.61
2018-01-03, 10:54:00, B,MAs:10489.95,MAl:10489.91
2018-01-03, 11:58:00, B,MAs:10480.63,MAl:10480.22
2018-01-03, 13:20:00, B,MAs:10498.83,MAl:10498.80
Final Portfolio Value: 100000.00


In [8]:
np.mean([1,2])

1.5